In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
# Pre-Processing for Hotels table

import pandas as pd

# read in data to dataframe
dfh = pd.read_csv('/content/drive/MyDrive/hotels.csv', encoding='latin-1')

# with open('hotels.csv', 'r') as f:
#     for i in range(100):
#         print(f.readline())

# cleaning column names, making lowecase, removing leading/trailing whitespace
dfh.columns = [i.strip().lower() for i in dfh.columns]

# creating a zipcode column, extracting zipcode from address field
dfh.loc[dfh.countycode=='US', 'zipcode'] = dfh.loc[dfh.countycode=='US'].pincode.str.split('-', expand=True)[0].apply(lambda x: str(x)[-6:])

# filtering for US only
dfh = dfh.loc[dfh.countycode=='US']

# creating a state column, extracting from citinyame field
dfh['state'] = dfh.cityname.str.split(',',expand=True)[1].str.strip()

# special case for Hawaii County
dfh.loc[dfh.state=='Hawaii County', 'state'] = 'Hawaii'

# creating reformatted city field
dfh['city'] = dfh.cityname.str.split(',',expand=True)[0].str.strip()

#
dfh['latitude'] = dfh['map'].str.split('|',expand=True)[0].astype(float)
dfh['longitude'] = dfh['map'].str.split('|',expand=True)[1].astype(float)


dfh = dfh[['hotelcode', 'latitude', 'longitude', 'city', 'citycode', 'state', 'zipcode', 'hotelname']]

dfh = dfh.drop_duplicates(subset=['hotelcode', 'citycode'])

print(dfh.shape[0])

dfh

139868


hotelcode   latitude   longitude                city  citycode  \
866143     1094195  29.980600  -92.114520           Abbeville    100328   
866144     1125999  29.973150  -92.112810           Abbeville    100328   
866145     1179258  29.967878  -92.085360           Abbeville    100328   
866146     1250024  29.968056  -92.084430           Abbeville    100328   
866147     1366117  29.981226  -92.124690           Abbeville    100328   
...            ...        ...         ...                 ...       ...   
1006006    1226889  42.435844  -87.823650                Zion    143364   
1006007    1365443  42.444513  -87.825511                Zion    143364   
1006008    1228497  37.980030  -78.213480     Zion Crossroads    142951   
1006009    1184658  37.270259 -112.874879  Zion National Park    143934   
1006010    1126355  44.301680  -92.685820            Zumbrota    142509   

             state zipcode                                  hotelname  
866143   Louisiana   70510                        Sunbelt Lodge Motel  
866144   Louisiana   70510                        Budget Inn & Suites  
866145   Louisiana   70510        Best Western Abbeville Inn & Suites  
866146   Louisiana   70510                       Motel 6 Abbeville LA  
866147   Louisiana   70510              Days Inn by Wyndham Abbeville  
...            ...     ...                                        ...  
1006006   Illinois   60099   Country Inn & Suites by Radisson Zion IL  
1006007   Illinois   60099                   The Inn at Market Square  
1006008   Virginia   22942  Best Western Plus Crossroads Inn & Suites  
1006009       Utah   84755                Zion Ponderosa Ranch Resort  
1006010  Minnesota   55992           Americas Best Value Inn Zumbrota  

[139868 rows x 8 columns]

In [40]:
# Step 1: Count occurrences of each hotelcode
hotelcode_counts = dfh['hotelcode'].value_counts()

# Step 2: Filter to show only hotelcodes with more than 2 occurrences
hotelcodes_more_than_twice = hotelcode_counts[hotelcode_counts > 2]

# Display hotelcodes that show up more than twice
print(hotelcodes_more_than_twice)

hotelcode
1307036    4
1258069    4
1002380    4
5504738    4
5504615    4
          ..
1052391    3
1104758    3
1228569    3
1097680    3
1007998    3
Name: count, Length: 1696, dtype: int64


In [51]:
dfh[dfh['hotelcode'] == 1307036]

hotelcode   latitude  longitude           city  citycode     state  \
872484    1307036  37.787173 -85.487287      Bardstown    110864  Kentucky   
873684    1307036  37.787173 -85.487287         Belton    111651  Missouri   
903212    1307036  37.787173 -85.487287  Elizabethtown    117672  Kentucky   
924806    1307036  37.787173 -85.487287    Kansas City    145881  Missouri   

       zipcode                 hotelname  
872484   40004  Howard Johnson Bardstown  
873684   40004  Howard Johnson Bardstown  
903212   40004  Howard Johnson Bardstown  
924806   40004  Howard Johnson Bardstown

In [ ]:
dfh[~dfh['zipcode'].str.isnumeric()]

hotelcode  latitude  longitude         city       state zipcode  \
866143     1094195  29.98060  -92.11452    Abbeville   Louisiana   70510   
866144     1125999  29.97315  -92.11281    Abbeville   Louisiana   70510   
866150     1784913  29.97754  -92.13370    Abbeville   Louisiana   70510   
866155     1203490  39.50333  -76.16816     Aberdeen    Maryland   21001   
866178     1098503  46.89556 -124.10500     Aberdeen  Washington   98595   
...            ...       ...        ...          ...         ...     ...   
1005998    1683319  28.27162  -82.17724  Zephyrhills     Florida   33540   
1005999    1683419  28.21837  -82.19493  Zephyrhills     Florida   33541   
1006000    1683475  28.20835  -82.19516  Zephyrhills     Florida   33541   
1006001    1683477  28.20827  -82.19138  Zephyrhills     Florida   33541   
1006002    1683587  28.20829  -82.19365  Zephyrhills     Florida   33541   

                              hotelname  
866143              Sunbelt Lodge Motel  
866144              Budget Inn & Suites  
866150            Ducote Williams House  
866155   Americas Best Value Inn Suites  
866178            Breakers Boutique Inn  
...                                 ...  
1005998           Baker Acres RV Resort  
1005999  Rainbow Village Of Zephyrhills  
1006000         Settlers Rest RV Resort  
1006001        Southern Charm RV Resort  
1006002            Sweetwater RV Resort  

[16451 rows x 7 columns]

In [55]:
# Save to csv
dfh.to_csv('hotels_cleaned_final.csv', index=False)

In [33]:
# Pre-Processing for Businesses table

# Import JSON module
import json
import pandas as pd

# Define JSON string

with open('/content/drive/MyDrive/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    d = f.readlines()

# jsonString = '{ "id": 121, "name": "Naveen", "course": "MERN Stack"}'

# jsonString = d[0]

# Convert JSON String to Python
business = []

for i in d:
    business.append(json.loads(i))

del d
df = pd.DataFrame(data=business)

df

business_id                      name  \
0       Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1       mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2       tUFrWirKiKi_TAnsVWINQQ                    Target   
3       MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4       mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
...                        ...                       ...   
150341  IUQopTMmYQG-qRtBk-8QnA              Binh's Nails   
150342  c8GjPIOTGVmIemT7j5_SyQ      Wild Birds Unlimited   
150343  _QAMST-NrQobXduilWEqSw         Claire's Boutique   
150344  mtGm22y5c2UHNXDFAjaPNw  Cyclery & Fitness Center   
150345  jV_XOycEzSlTx-65W906pg                   Sic Ink   

                                address           city state postal_code  \
0                1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1       87 Grasso Plaza Shopping Center         Affton    MO       63123   
2                  5255 E Broadway Blvd         Tucson    AZ       85711   
3                           935 Race St   Philadelphia    PA       19107   
4                         101 Walnut St     Green Lane    PA       18054   
...                                 ...            ...   ...         ...   
150341                3388 Gateway Blvd       Edmonton    AB     T6J 5H2   
150342               2813 Bransford Ave      Nashville    TN       37204   
150343           6020 E 82nd St, Ste 46   Indianapolis    IN       46250   
150344                     2472 Troy Rd   Edwardsville    IL       62025   
150345            238 Apollo Beach Blvd   Apollo beach    FL       33572   

         latitude   longitude  stars  review_count  is_open  \
0       34.426679 -119.711197    5.0             7        0   
1       38.551126  -90.335695    3.0            15        1   
2       32.223236 -110.880452    3.5            22        0   
3       39.955505  -75.155564    4.0            80        1   
4       40.338183  -75.471659    4.5            13        1   
...           ...         ...    ...           ...      ...   
150341  53.468419 -113.492054    3.0            13        1   
150342  36.115118  -86.766925    4.0             5        1   
150343  39.908707  -86.065088    3.5             8        1   
150344  38.782351  -89.950558    4.0            24        1   
150345  27.771002  -82.394910    4.5             9        1   

                                               attributes  \
0                           {'ByAppointmentOnly': 'True'}   
1                  {'BusinessAcceptsCreditCards': 'True'}   
2       {'BikeParking': 'True', 'BusinessAcceptsCredit...   
3       {'RestaurantsDelivery': 'False', 'OutdoorSeati...   
4       {'BusinessAcceptsCreditCards': 'True', 'Wheelc...   
...                                                   ...   
150341  {'ByAppointmentOnly': 'False', 'RestaurantsPri...   
150342  {'BusinessAcceptsCreditCards': 'True', 'Restau...   
150343  {'RestaurantsPriceRange2': '1', 'BusinessAccep...   
150344  {'BusinessParking': '{'garage': False, 'street...   
150345  {'WheelchairAccessible': 'True', 'BusinessAcce...   

                                               categories  \
0       Doctors, Traditional Chinese Medicine, Naturop...   
1       Shipping Centers, Local Services, Notaries, Ma...   
2       Department Stores, Shopping, Fashion, Home & G...   
3       Restaurants, Food, Bubble Tea, Coffee & Tea, B...   
4                               Brewpubs, Breweries, Food   
...                                                   ...   
150341                         Nail Salons, Beauty & Spas   
150342  Pets, Nurseries & Gardening, Pet Stores, Hobby...   
150343  Shopping, Jewelry, Piercing, Toy Stores, Beaut...   
150344  Fitness/Exercise Equipment, Eyewear & Optician...   
150345  Beauty & Spas, Permanent Makeup, Piercing, Tattoo   

                                                    hours  
0                                                    None  
1       {'Monday': '0:0-0:0', 'Tuesday': '

In [10]:
df = df[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude']]
df = df.rename(columns={'postal_code': 'zipcode'})
df


business_id                      name  \
0       Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1       mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2       tUFrWirKiKi_TAnsVWINQQ                    Target   
3       MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4       mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
...                        ...                       ...   
150341  IUQopTMmYQG-qRtBk-8QnA              Binh's Nails   
150342  c8GjPIOTGVmIemT7j5_SyQ      Wild Birds Unlimited   
150343  _QAMST-NrQobXduilWEqSw         Claire's Boutique   
150344  mtGm22y5c2UHNXDFAjaPNw  Cyclery & Fitness Center   
150345  jV_XOycEzSlTx-65W906pg                   Sic Ink   

                                address           city state  zipcode  \
0                1616 Chapala St, Ste 2  Santa Barbara    CA    93101   
1       87 Grasso Plaza Shopping Center         Affton    MO    63123   
2                  5255 E Broadway Blvd         Tucson    AZ    85711   
3                           935 Race St   Philadelphia    PA    19107   
4                         101 Walnut St     Green Lane    PA    18054   
...                                 ...            ...   ...      ...   
150341                3388 Gateway Blvd       Edmonton    AB  T6J 5H2   
150342               2813 Bransford Ave      Nashville    TN    37204   
150343           6020 E 82nd St, Ste 46   Indianapolis    IN    46250   
150344                     2472 Troy Rd   Edwardsville    IL    62025   
150345            238 Apollo Beach Blvd   Apollo beach    FL    33572   

         latitude   longitude  
0       34.426679 -119.711197  
1       38.551126  -90.335695  
2       32.223236 -110.880452  
3       39.955505  -75.155564  
4       40.338183  -75.471659  
...           ...         ...  
150341  53.468419 -113.492054  
150342  36.115118  -86.766925  
150343  39.908707  -86.065088  
150344  38.782351  -89.950558  
150345  27.771002  -82.394910  

[150346 rows x 8 columns]

In [11]:
# Save to csv
df.to_csv('yelp_business_cleaned.csv', index=False)

In [22]:
# Pre-Processing for Categories table

import pandas as pd
import json
import uuid

# Step 1: Initialize an empty set to store unique categories
categories_set = set()

# Step 2: Read the JSON file line by line
with open('/content/drive/MyDrive/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
        for line in f:
          # Parse each line as a JSON object
          business = json.loads(line.strip())

          # Check if "categories" is a non-null and valid string
          if isinstance(business.get("categories"), str) and business["categories"].strip():
              # Split categories string by commas, strip whitespace, and add each category to the set
              categories = [category.strip() for category in business["categories"].split(",")]
              categories_set.update(categories)

print(len(categories_set))
print(categories_set)

# Step 3: Create a DataFrame for unique categories with IDs
categories_df = pd.DataFrame({
    'category_id': [str(uuid.uuid4()) for _ in categories_set],  # Generate unique IDs
    'category': list(categories_set)})

print(categories_df.shape[0])
categories_df.head(20)


1311
{'Scandinavian', 'Art Restoration', 'Health Retreats', 'Oil Change Stations', 'Allergists', 'Television Stations', 'Music Production Services', 'Rolfing', 'Cigar Bars', 'Dance Studios', 'Stucco Services', 'Food', 'Doctors', 'Sugaring', 'Wedding Planning', 'Prenatal/Perinatal Care', 'Auto Insurance', 'Musical Instrument Services', 'Arabic', 'Circus Schools', 'Diving', 'Oncologist', 'Divorce & Family Law', 'Excavation Services', 'Painters', 'Drive-Thru Bars', 'Holiday Decorations', 'Pumpkin Patches', 'Festivals', 'Pet Training', 'Walking Tours', 'Parklets', 'Parking', 'Life Insurance', 'Trailer Repair', 'Yoga', 'Auto Detailing', 'Behavior Analysts', 'Boxing', 'Scooter Tours', 'Mass Media', 'Dance Schools', 'Photography Stores & Services', 'Retirement Homes', 'Outdoor Furniture Stores', 'Tanning', 'Diagnostic Services', 'Concierge Medicine', 'Elder Care Planning', 'Hybrid Car Repair', 'Caribbean', 'Sports Psychologists', 'Cycling Classes', 'Japanese', 'Makeup Artists', 'Chicken Wings

category_id                     category
0   9e573e90-def5-4530-b352-99c848c0db6a                 Scandinavian
1   5e67f5cd-1ce5-4327-b3e7-681301b4bcfe              Art Restoration
2   cd50db21-da41-47ef-a362-45f343bb14b7              Health Retreats
3   4633b68f-9145-4202-9886-0c8deaf0b05f          Oil Change Stations
4   8c31b799-cbda-4b1a-875d-32b8481acf99                   Allergists
5   35ddefad-a38e-4816-96fe-525e3f377e44          Television Stations
6   d2cb597b-cbce-4d65-9034-db77aa098408    Music Production Services
7   b9bbcfbe-deaa-4e00-b953-364b398496f0                      Rolfing
8   9b358aba-eab6-49a6-b062-df8e178a8967                   Cigar Bars
9   3787ac17-489d-4820-8890-45e1d5c85e73                Dance Studios
10  73c95e02-7d3c-4736-85f3-5e145b545d44              Stucco Services
11  2ba55eba-3c48-4677-a03f-51d0e86e7a1f                         Food
12  63ea0b47-3c80-412f-bee7-50c58012e55a                      Doctors
13  969e2543-5cfb-4529-961b-9cf43474eacc                     Sugaring
14  40786816-4825-4803-929f-500a4787704c             Wedding Planning
15  3d03e675-a050-4f48-b6de-35e55b1b4187      Prenatal/Perinatal Care
16  b3a3d656-aac0-46f7-86dd-b5310375ba5b               Auto Insurance
17  0bc06680-0525-4c0b-88ea-bec9e3141fce  Musical Instrument Services
18  3d1c23be-0498-4b3c-b357-5923e2b025e8                       Arabic
19  8599dfaf-db00-4bde-a51f-34a358e740ec               Circus Schools

In [25]:
# Save to csv
categories_df.to_csv('categories_cleaned.csv', index=False)

In [24]:
# Pre-Processing for isCategory table

import pandas as pd
import json

# Step 1: Load the business data
business_data = []
with open('/content/drive/MyDrive/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    for line in f:
        business = json.loads(line.strip())
        if "business_id" in business and "categories" in business and isinstance(business["categories"], str) and business["categories"].strip():
            # Split categories by comma and strip whitespace
            categories = [category.strip() for category in business["categories"].split(",")]
            for category in categories:
                business_data.append((business["business_id"], category))

# Convert to DataFrame
isCategory_df = pd.DataFrame(business_data, columns=["business_id", "category"])

# Step 2: Load categories_df, which contains unique categories and their IDs
# Ensure categories_df is loaded as per previous steps
# categories_df = pd.read_csv('categories.csv')  # Uncomment if loading from CSV

# Step 3: Merge isCategory_df with categories_df to get the category_id
isCategory_df = isCategory_df.merge(categories_df, on="category", how="left")
isCategory_df = isCategory_df[['business_id', 'category_id', 'category']]
isCategory_df = isCategory_df.drop_duplicates()
print(isCategory_df.shape[0])
isCategory_df


668549


business_id                           category_id  \
0       Pns2l4eNsfO8kk83dixA6A  63ea0b47-3c80-412f-bee7-50c58012e55a   
1       Pns2l4eNsfO8kk83dixA6A  75aceb2c-5df1-495e-9ac5-1d39499f0a1d   
2       Pns2l4eNsfO8kk83dixA6A  26b27a2e-af5c-4ce2-bd4a-2cada53de524   
3       Pns2l4eNsfO8kk83dixA6A  0d463bf7-d36c-4f3c-920b-fb57784b75ec   
4       Pns2l4eNsfO8kk83dixA6A  0f578b84-6f47-46a3-920d-fcd25153d482   
...                        ...                                   ...   
668587  mtGm22y5c2UHNXDFAjaPNw  44243c81-579d-491c-bb7d-cf23a749b103   
668588  jV_XOycEzSlTx-65W906pg  3ba765bf-2dda-4c95-9790-3a15565b86db   
668589  jV_XOycEzSlTx-65W906pg  e9d3f864-0a44-4b0f-afca-b5fd678e2ca9   
668590  jV_XOycEzSlTx-65W906pg  328015ee-c2f8-4059-a1c3-997150c7276a   
668591  jV_XOycEzSlTx-65W906pg  007b2c12-3e9b-400a-84a9-acf0daef058f   

                            category  
0                            Doctors  
1       Traditional Chinese Medicine  
2              Naturopathic/Holistic  
3                        Acupuncture  
4                   Health & Medical  
...                              ...  
668587                         Bikes  
668588                 Beauty & Spas  
668589              Permanent Makeup  
668590                      Piercing  
668591                        Tattoo  

[668549 rows x 3 columns]

In [26]:
# Save to csv
isCategory_df.to_csv('isCategory_cleaned.csv', index=False)